# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298279639767                   -0.85    5.0         
  2   -8.300170704222       -2.72       -1.25    1.0    119ms
  3   -8.300429372303       -3.59       -1.89    2.2    140ms
  4   -8.300462584312       -4.48       -2.76    3.1    157ms
  5   -8.300464406639       -5.74       -3.17    2.8    241ms
  6   -8.300464524193       -6.93       -3.33   12.2    295ms
  7   -8.300464577993       -7.27       -3.47    2.4    146ms
  8   -8.300464611486       -7.48       -3.62    1.5    129ms
  9   -8.300464632371       -7.68       -3.79    1.4    131ms
 10   -8.300464640219       -8.11       -3.97    5.0    255ms
 11   -8.300464643567       -8.48       -4.27    4.5    217ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67434011537                   -0.70    5.8         
  2   -16.67861188936       -2.37       -1.14    1.2    322ms
  3   -16.67921784826       -3.22       -1.87    3.4    416ms
  4   -16.67928005046       -4.21       -2.77    3.8    398ms
  5   -16.67928603406       -5.22       -3.18    4.9    478ms
  6   -16.67928620418       -6.77       -3.50    2.9    390ms
  7   -16.67928621928       -7.82       -3.95    1.6    305ms
  8   -16.67928622157       -8.64       -4.58    2.5    340ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32561967589                   -0.56    7.1         
  2   -33.33282300086       -2.14       -1.00    1.0    1.12s
  3   -33.33409053444       -2.90       -1.74    3.5    1.34s
  4   -33.33428332863       -3.71       -2.64    4.1    1.37s
  5   -33.33693823205       -2.58       -2.49    7.4    1.93s
  6   -33.33693847343       -6.62       -2.49    5.2    1.37s
  7   -33.33693840450   +   -7.16       -2.49    1.0    1.07s
  8   -33.33693810745   +   -6.53       -2.51    1.9    1.15s
  9   -33.33683041532   +   -3.97       -2.04    2.2    1.22s
 10   -33.33628981926   +   -3.27       -1.78    2.8    1.45s
 11   -33.33646684932       -3.75       -1.83    1.1    1.06s
 12   -33.33676286815       -3.53      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298557932600                   -0.85    5.0         
  2   -8.300280692061       -2.76       -1.59    1.0   98.0ms
  3   -8.300428818990       -3.83       -2.53    2.4    120ms
  4   -8.300371003286   +   -4.24       -2.27    3.2    161ms
  5   -8.300464152107       -4.03       -3.41    1.1    100ms
  6   -8.300464535336       -6.42       -3.72    2.5    181ms
  7   -8.300464635898       -7.00       -4.15    1.1   97.8ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32593889871                   -0.56    7.2         
  2   -33.29173893084   +   -1.47       -1.26    1.2    937ms
  3   +6.358339748175   +    1.60       -0.27    6.8    2.28s
  4   -33.33381194581        1.60       -1.97    5.8    1.95s
  5   -33.28261589138   +   -1.29       -1.46    4.1    1.73s
  6   -33.17249707105   +   -0.96       -1.44    4.2    1.56s
  7   -33.13359766389   +   -1.41       -1.40    5.6    1.67s
  8   -33.33607638471       -0.69       -2.32    4.4    1.48s
  9   -33.33339116449   +   -2.57       -2.18    2.5    1.25s
 10   -33.33653446001       -2.50       -2.63    2.2    1.12s
 11   -33.33689732920       -3.44       -3.10    3.5    1.19s
 12   -33.33693975556       -4.37      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.